In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../datasrc/HIST_PAINEL_COVIDBR_31mai2020.xlsx')

In [3]:
df.head(5)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos
0,Brasil,NaN,NaN,76.0,NaN,NaN,NaN,2020-02-26,9.0,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN
1,Brasil,NaN,NaN,76.0,NaN,NaN,NaN,2020-02-27,9.0,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN
2,Brasil,NaN,NaN,76.0,NaN,NaN,NaN,2020-02-28,9.0,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN
3,Brasil,NaN,NaN,76.0,NaN,NaN,NaN,2020-02-29,9.0,210147125.0,2.0,1.0,0.0,0.0,NaN,NaN
4,Brasil,NaN,NaN,76.0,NaN,NaN,NaN,2020-03-01,10.0,210147125.0,2.0,0.0,0.0,0.0,NaN,NaN


In [4]:
df.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019',
       'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos',
       'Recuperadosnovos', 'emAcompanhamentoNovos'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138771 entries, 0 to 138770
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   regiao                 138733 non-null  object 
 1   estado                 138636 non-null  object 
 2   municipio              135796 non-null  object 
 3   coduf                  138733 non-null  float64
 4   codmun                 136470 non-null  float64
 5   codRegiaoSaude         135796 non-null  float64
 6   nomeRegiaoSaude        135796 non-null  object 
 7   data                   138733 non-null  object 
 8   semanaEpi              138733 non-null  float64
 9   populacaoTCU2019       138059 non-null  float64
 10  casosAcumulado         138733 non-null  float64
 11  casosNovos             138733 non-null  float64
 12  obitosAcumulado        138733 non-null  float64
 13  obitosNovos            138733 non-null  float64
 14  Recuperadosnovos       44 non-null  

In [6]:
df['estado'].unique()

array([nan, 'RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE',
       'RN', 'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR',
       'SC', 'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

## Base de dados de longitude e latitude

In [7]:
df_localizacoes = pd.read_csv('../datasets/datasets_localizacao_municipios.csv')

In [8]:
df_localizacoes.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [9]:
df_localizacoes['codmun'] = df_localizacoes['codigo_ibge']//10
df_localizacoes = df_localizacoes[['codmun', 'latitude', 'longitude']]
df_localizacoes.head()

,codmun,latitude,longitude
0,520005,-16.75730,-49.4412
1,310010,-18.48310,-47.3916
2,520010,-16.19700,-48.7057
3,310020,-19.15510,-45.4444
4,150010,-1.72183,-48.8788


## Divide as base de dados (cidades e estados)

Primeiramente vamos extrair a base apenas de municípios.

In [10]:
df_city = df[df['municipio'].notnull()]

In [11]:
df_city = df_city.fillna(0)

In [12]:
df_city = pd.merge(df_city, df_localizacoes, on='codmun')

estados = {'Acre': 'AC',
           'Alagoas': 'AL',
           'Amapá': 'AP',
           'Amazonas': 'AM',
           'Bahia': 'BA',
           'Ceará': 'CE',
           'Distrito Federal': 'DF',
           'Espírito Santo': 'ES',
           'Goiás': 'GO',
           'Maranhão': 'MA',
           'Mato Grosso': 'MT',
           'Mato Grosso do Sul': 'MS',
           'Minas Gerais': 'MG',
           'Pará': 'PA',
           'Paraíba': 'PB',
           'Paraná': 'PR',
           'Pernambuco': 'PE',
           'Piauí': 'PI',
           'Rio de Janeiro': 'RJ',
           'Rio Grande do Norte': 'RN',
           'Rio Grande do Sul': 'RS',
           'Rondônia': 'RO',
           'Roraima': 'RR',
           'Santa Catarina': 'SC',
           'São Paulo': 'SP',
           'Sergipe': 'SE',
           'Tocantins': 'TO'} 

df_estados_siglas = pd.DataFrame(list(estados.items()), columns='uf estado'.split())
df_city = pd.merge(df_city, df_estados_siglas, on='estado')

In [13]:
df_city.head(5)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,latitude,longitude,uf
0,Norte,RO,Alta Floresta D'Oeste,11.0,110001.0,11005.0,Zona da Mata,2020-05-30,23.0,22945.0,3.0,0.0,0.0,0.0,0.0,0.0,-11.9283,-61.9953,Rondônia
1,Norte,RO,Alta Floresta D'Oeste,11.0,110001.0,11005.0,Zona da Mata,2020-05-02,18.0,22945.0,1.0,0.0,0.0,0.0,0.0,0.0,-11.9283,-61.9953,Rondônia
2,Norte,RO,Alta Floresta D'Oeste,11.0,110001.0,11005.0,Zona da Mata,2020-05-03,19.0,22945.0,1.0,0.0,0.0,0.0,0.0,0.0,-11.9283,-61.9953,Rondônia
3,Norte,RO,Alta Floresta D'Oeste,11.0,110001.0,11005.0,Zona da Mata,2020-05-04,19.0,22945.0,1.0,0.0,0.0,0.0,0.0,0.0,-11.9283,-61.9953,Rondônia
4,Norte,RO,Alta Floresta D'Oeste,11.0,110001.0,11005.0,Zona da Mata,2020-05-05,19.0,22945.0,1.0,0.0,0.0,0.0,0.0,0.0,-11.9283,-61.9953,Rondônia


Agora vamos extrair a base de estados apenas com as colunas de interesse.

In [14]:
df_state = df[(df['municipio'].isnull()) & (df['estado'].notnull())]

In [15]:
df_state = df_state.drop(['municipio', 
                          'codmun', 
                          'codRegiaoSaude', 
                          'nomeRegiaoSaude', 
                          'emAcompanhamentoNovos'], axis=1)

df_state = df_state.fillna(0)

In [16]:
df_state.head()

,regiao,estado,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos
219,Norte,RR,14.0,2020-05-30,23.0,0.0,37.0,0.0,1.0,0.0,0.0
1034,Nordeste,RN,24.0,2020-05-30,23.0,0.0,25.0,0.0,3.0,0.0,0.0
1373,Nordeste,PE,26.0,2020-05-30,23.0,0.0,50.0,0.0,9.0,0.0,0.0
1553,Nordeste,AL,27.0,2020-05-30,23.0,0.0,50.0,0.0,3.0,0.0,0.0
1727,Nordeste,BA,29.0,2020-05-30,23.0,0.0,93.0,0.0,3.0,0.0,0.0


In [32]:
df_country = df[(df['municipio'].isnull()) & (df['estado'].isnull()) & (df['regiao'].notnull())]

In [33]:
df_country = df_country.drop(['municipio', 
                              'codmun', 
                              'estado',
                              'codRegiaoSaude', 
                              'nomeRegiaoSaude', 
                              'emAcompanhamentoNovos'], axis=1)

df_country = df_country.fillna(0)
df_country['casosPorDia'] = df_country['casosAcumulado'].diff().fillna(0)
df_country['obitosPorDia'] = df_country['obitosAcumulado'].diff().fillna(0)

In [34]:
df_country.head()

,regiao,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,casosPorDia,obitosPorDia
0,Brasil,76.0,2020-02-26,9.0,210147125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Brasil,76.0,2020-02-27,9.0,210147125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brasil,76.0,2020-02-28,9.0,210147125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brasil,76.0,2020-02-29,9.0,210147125.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0
4,Brasil,76.0,2020-03-01,10.0,210147125.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_country

,regiao,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,casosPorDia,obitosPorDia
0,Brasil,76.0,2020-02-26,9.0,210147125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Brasil,76.0,2020-02-27,9.0,210147125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brasil,76.0,2020-02-28,9.0,210147125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brasil,76.0,2020-02-29,9.0,210147125.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0
4,Brasil,76.0,2020-03-01,10.0,210147125.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
92,Brasil,76.0,2020-05-28,22.0,210147125.0,438238.0,26417.0,26754.0,1156.0,177604.0,26417.0,1156.0
93,Brasil,76.0,2020-05-29,22.0,210147125.0,465166.0,26928.0,27878.0,1124.0,189476.0,26928.0,1124.0
4218,Brasil,76.0,2020-05-30,22.0,210147125.0,498440.0,33274.0,28834.0,956.0,200892.0,33274.0,956.0
4219,Brasil,76.0,2020-05-31,23.0,210147125.0,514849.0,16409.0,29314.0,480.0,206555.0,16409.0,480.0


Vamos salvar os dataframes em arquivo excel para leitura no PowerBI.

In [113]:
df_city.to_excel('../datasets/dataset_covid_city.xlsx')

In [114]:
df_state.to_excel('../datasets/dataset_covid_state.xlsx')

In [115]:
df_country.to_excel('../datasets/dataset_covid_country.xlsx')

## Sandbox

In [10]:
from pathlib import Path

def most_recent_file(path):
  # Identifica as datas de modificação
  data_modificacao = lambda f: f.stat().st_mtime

  # Encontra o diretório no qual estão os arquivos
  directory = Path(path)

  # Seleciona apenas os arquivos com extensão .xlsx
  files = directory.glob('*.xlsx')

  # Ordena os arquivos do mais recente para o menos recente
  sorted_files = sorted(files, key=data_modificacao, reverse=True)

  # retorna o arquivo mais recente
  return sorted_files[0]

In [12]:
PATH = '../datasrc/'

df = pd.read_excel(most_recent_file(PATH))

In [15]:
str(most_recent_file(PATH))

'..\\datasrc\\HIST_PAINEL_COVIDBR_24mai2020 (2).xlsx'